## Get Next Experimental Trials

In [15]:
import os
import numpy as np
import pandas as pd
from ax.service.ax_client import AxClient, ObjectiveProperties

# hide future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### Set up the experiment and specify parameter types and value ranges

In [16]:
ax_client = AxClient()

# define the experiment and objectives
ax_client.create_experiment(
    parameters=[
        {"name": "PINK", "type": "range", "bounds": [0, 20], "value_type": "int"},
        {"name": "GREEN", "type": "range", "bounds": [0, 20], "value_type": "int"},
        {"name": "BLUE", "type": "range", "bounds": [0, 20], "value_type": "int"},
        {"name": "PURPLE", "type": "range", "bounds": [0, 20], "value_type": "int"},
        {"name": "SOLID_LIQUID_RATIO", "type": "range", "bounds": [0.5, 2.0], "value_type": "float", "digits": 1},
    ],
    objectives={
        "COLOR_ERR": ObjectiveProperties(minimize=True, threshold=1e-3),
        "STRAIN_ERR": ObjectiveProperties(minimize=True, threshold=1e-3),
    },
    choose_generation_strategy_kwargs={"num_initialization_trials": 0},
)

[INFO 04-06 09:59:16] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.
[INFO 04-06 09:59:16] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[RangeParameter(name='PINK', parameter_type=INT, range=[0, 20]), RangeParameter(name='GREEN', parameter_type=INT, range=[0, 20]), RangeParameter(name='BLUE', parameter_type=INT, range=[0, 20]), RangeParameter(name='PURPLE', parameter_type=INT, range=[0, 20]), RangeParameter(name='SOLID_LIQUID_RATIO', parameter_type=FLOAT, range=[0.5, 2.0], digits=1)], parameter_constraints=[]).
[INFO 04-06 09:59:16] ax.modelbridge.dispatch_utils: Using Models.BOTORCH_MODULAR since there is at least one ordered parameter and there are no unordered categorical parameters.
[INFO 04-06 09:59:16] ax.modelbridge.dispatch_utils: Calculating the number of remaining initialization trials based o

### Load in the previous experimental results, train a model, and predict the next set of trials

In [17]:
# instantiate a running_experiment file if it doesn't exist
if not os.path.exists(os.path.join('data', 'running_trials.csv')):
    try:
        os.system("cp data/sobol_trials.csv data/running_trials.csv")
    except:
        os.system("copy data\\sobol_trials.csv data\\running_trials.csv")

# read in the previous trials data and correct the data types
running_trials = pd.read_csv(os.path.join('data', 'running_trials.csv')).astype({'PINK': int, 'GREEN': int, 'BLUE': int, 'PURPLE': int})

# separate parameters from results
X_train = running_trials.iloc[:,:5].to_dict(orient="records")
y_train = running_trials.iloc[:,5:]#.to_dict(orient="records")
y_train = [{k:v for (k,v) in zip(["COLOR_ERR", "STRAIN_ERR"], [(row[0], row[1]), (row[2], row[3])])} for row in y_train.values]

# add the previous trials to the experiment
for i in range(len(running_trials)):
    parameterization = X_train[i]
    ax_client.attach_trial(parameterization)
    ax_client.complete_trial(trial_index=i, raw_data=y_train[i])

# generate new trials
batch_size = 5
parameterizations, optimization_complete = ax_client.get_next_trials(batch_size)
trials_df = ax_client.get_trials_data_frame()
display(trials_df)

[INFO 04-06 09:59:16] ax.core.experiment: Attached custom parameterizations [{'PINK': 9, 'GREEN': 12, 'BLUE': 10, 'PURPLE': 6, 'SOLID_LIQUID_RATIO': 1.9}] as trial 0.
[INFO 04-06 09:59:16] ax.service.ax_client: Completed trial 0 with data: {'COLOR_ERR': (0.1, 0.1), 'STRAIN_ERR': (0.1, 0.1)}.
[INFO 04-06 09:59:16] ax.core.experiment: Attached custom parameterizations [{'PINK': 12, 'GREEN': 8, 'BLUE': 19, 'PURPLE': 17, 'SOLID_LIQUID_RATIO': 0.5}] as trial 1.
[INFO 04-06 09:59:16] ax.service.ax_client: Completed trial 1 with data: {'COLOR_ERR': (0.2, 0.2), 'STRAIN_ERR': (0.2, 0.2)}.
[INFO 04-06 09:59:16] ax.core.experiment: Attached custom parameterizations [{'PINK': 19, 'GREEN': 16, 'BLUE': 4, 'PURPLE': 3, 'SOLID_LIQUID_RATIO': 1.1}] as trial 2.
[INFO 04-06 09:59:16] ax.service.ax_client: Completed trial 2 with data: {'COLOR_ERR': (0.3, 0.3), 'STRAIN_ERR': (0.3, 0.3)}.
[INFO 04-06 09:59:16] ax.core.experiment: Attached custom parameterizations [{'PINK': 0, 'GREEN': 5, 'BLUE': 13, 'PURPLE

,trial_index,arm_name,trial_status,generation_method,is_feasible,COLOR_ERR,STRAIN_ERR,PINK,GREEN,BLUE,PURPLE,SOLID_LIQUID_RATIO
0,0,0_0,COMPLETED,Manual,True,0.1,0.1,9,12,10,6,1.9
1,1,1_0,COMPLETED,Manual,True,0.2,0.2,12,8,19,17,0.5
2,2,2_0,COMPLETED,Manual,True,0.3,0.3,19,16,4,3,1.1
3,3,3_0,COMPLETED,Manual,True,0.4,0.4,0,5,13,14,1.5
4,4,4_0,COMPLETED,Manual,True,0.5,0.5,3,18,15,11,1.4
5,5,5_0,RUNNING,BoTorch,NaN,NaN,NaN,10,10,10,3,2.0
6,6,6_0,RUNNING,BoTorch,NaN,NaN,NaN,6,14,5,3,2.0
7,7,7_0,RUNNING,BoTorch,NaN,NaN,NaN,17,7,18,6,2.0


## Next Trials To Run

In [18]:
display(trials_df.iloc[-3:, :])

,trial_index,arm_name,trial_status,generation_method,is_feasible,COLOR_ERR,STRAIN_ERR,PINK,GREEN,BLUE,PURPLE,SOLID_LIQUID_RATIO
5,5,5_0,RUNNING,BoTorch,NaN,NaN,NaN,10,10,10,3,2.0
6,6,6_0,RUNNING,BoTorch,NaN,NaN,NaN,6,14,5,3,2.0
7,7,7_0,RUNNING,BoTorch,NaN,NaN,NaN,17,7,18,6,2.0
